In [51]:
import pandas as pd
import requests
from glob import glob
import pdfplumber
import os

In [52]:
def parsing(route):


    # API 엔드포인트 및 API 키
    url = "https://api.upstage.ai/v1/document-ai/document-parse"
    api_key = "up_1Nau8lF94VG6q2SuDtNqM4SzOtXGd"  # 실제 API 키로 대체하세요

    # 파일 경로 설정
    file_path = route

    # HTTP 요청 헤더 및 파일
    headers = {
        "Authorization": f"Bearer {api_key}"
    }
    files = {
        "document": open(file_path, "rb")  # 파일을 바이너리로 읽어 업로드
    }

    # POST 요청 전송
    response = requests.post(url, headers=headers, files=files)

    # 결과 확인
    if response.status_code == 200:
        print(f"{file_path.split('/')[-1]} 파싱 성공 !")
    else:
        print(f"{file_path.split('/')[-1]} 파싱 중 오류 발생: {response.status_code}")
    return response.json()


In [53]:


def crop_and_save(route, page_num, coords, id, dpi=500, scale = 15):
    with pdfplumber.open(route) as pdf:
        page = pdf.pages[page_num-1]
    dir_route = f'datas/{route.split('/')[-1][:-4]}'
    save_route = dir_route + f'/page_{page_num}_id_{id}_image.png'
    os.makedirs(dir_route, exist_ok=True)
    page_width = page.width
    page_height = page.height

    # 상대 좌표를 절대 픽셀 좌표로 변환
    x0 = coords[0]['x'] * page_width
    y0 = max(0, coords[0]['y'] * page_height - scale)
    x1 = coords[2]['x'] * page_width
    y1 = min(page_height, coords[2]['y'] * page_height + scale)

    # 크롭 영역 정의
    crop_box = (x0, y0, x1, y1)
    page = page.crop(crop_box).to_image(resolution = dpi)
    page.save(save_route)

    return save_route

In [54]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import base64
load_dotenv()

def image_summarization(image_path, investment, company):
    api_key = os.environ['OPENAI_API_KEY']
    client = OpenAI(api_key=api_key)
    # Open the image file and encode it as a base64 string
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")

    base64_image = encode_image(image_path)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {
                "role": "system", "content": 

                "당신은 RAG 시스템을 위한 이미지 요약가입니다."

                },
                {"role": "user", "content": [{"type": "text", "text": 
                                            
                f"""
                다음은 {investment}의 {company}에 대한 리포트 중 일부입니다.
                주어진 이미지가 무엇을 나타내고 있는지 상세히 기술하세요.
                회사 이름과 그래프를 작성한 투자증권을 명확히 밝히세요.
                마크다운 문법으로 작성하세요.
                """

                },
                {"type": "image_url", "image_url": {
                "url": f"data:image/png;base64,{base64_image}"}
                }
            ]}
        ],
        temperature=0.0,
    )

    result = response.choices[0].message.content + '\n' + f'이미지 경로 : {image_path}'
    return result

In [55]:
def make_md(route, elements):
    md_file = f'datas/{route.split('/')[-1][:-4]}/{route.split('/')[-1][:-4]}.md'
    dir_route = f'datas/{route.split('/')[-1][:-4]}'
    os.makedirs(dir_route, exist_ok=True)
    company_name = route.split('/')[-1].split('_')[0]
    investment = route.split('/')[-1].split('_')[1][:-4]
    images = ['table', 'figure']
    # result['elements']를 순회하며 처리
    for item in elements:
        with open(md_file, 'a', encoding='utf-8') as md:
            if item['category'] in images:
                # 이미지 루트를 반환하는 crop_and_save 함수 호출
                image_route = crop_and_save(route, item['page'], item['coordinates'], item['id'])
                # 이미지 경로를 Markdown 형식으로 추가
                md.write(f"![{item['category']}]({image_route.split('/')[-1]})\n\n\n")
                result = image_summarization(image_route, company_name, investment)
                md.write(f"{result}\n\n")
            else:
                # 이미지가 아닌 경우 html 콘텐츠를 추가
                html_content = item['content'].get('html', '')
                md.write(f"{html_content}\n\n\n")


In [56]:

data_routes = glob('datas/*')
route = '/data/ephemeral/home/PDF Parsing/datas/카카오뱅크_키움증권(2022.10.12).pdf'
result = parsing(route)
make_md(route, result['elements'])


카카오뱅크_키움증권(2022.10.12).pdf 파싱 성공 !
